In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.utils import to_categorical
import time

# Load and preprocess the data
def load_data():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize pixel values

    # One-hot encode the labels
    y_train_one_hot = to_categorical(y_train, num_classes=10)
    y_test_one_hot = to_categorical(y_test, num_classes=10)

    return x_train, y_train_one_hot, x_test, y_test_one_hot

# Build the neural network model
def build_model():
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(64, activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='SGD',
                  loss='categorical_crossentropy',  # Changed loss function
                  metrics=['accuracy'])
    return model

# Train and evaluate the model
def train_and_evaluate_model(model, x_train, y_train, x_test, y_test, epochs=100, log_file='training_log_tf.csv'):
    # Initialize CSVLogger
    csv_logger = CSVLogger(log_file, append=True)

    start_time = time.time()
    model.fit(
        x_train,
        y_train,
        epochs=epochs,
        batch_size=64,
        verbose=2,  # You can set to 1 or 2 for different verbosity levels
        callbacks=[csv_logger]
    )
    end_time = time.time()
    
    # Evaluate on test data
    loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"Test Loss: {loss:.4f}")
    print(f"Execution time: {end_time - start_time:.4f} seconds")
    return test_accuracy

# Main function to run the pipeline
def main():
    x_train, y_train, x_test, y_test = load_data()
    model = build_model()
    train_and_evaluate_model(model, x_train, y_train, x_test, y_test)

if __name__ == "__main__":
    main()


In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.utils import to_categorical
import time
import csv
import os
import uuid

# Load and preprocess the data
def load_data():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize pixel values

    # One-hot encode the labels
    y_train_one_hot = to_categorical(y_train, num_classes=10)
    y_test_one_hot = to_categorical(y_test, num_classes=10)

    return x_train, y_train_one_hot, x_test, y_test_one_hot

# Build the neural network model
def build_model():
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(256, activation='relu'),
        Dense(64, activation='relu'),
        Dense(10, activation='sigmoid')
    ])
    model.compile(optimizer='SGD',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Custom Callback to log extended metrics
class ExtendedCSVLogger(Callback):
    def __init__(self, filename, append=False):
        super(ExtendedCSVLogger, self).__init__()
        self.filename = filename
        self.append = append
        self.start_time = None
        self.batches_per_epoch = 0

        # Initialize CSV file with headers if not appending or file doesn't exist
        if not append or not os.path.exists(filename):
            with open(self.filename, 'w', newline='') as f:
                writer = csv.writer(f)
                headers = ['Epoch', 'Progress%', 'Loss', 'Accuracy%', 'Epoch Time(s)', 'Batches/s']
                writer.writerow(headers)

    def on_epoch_begin(self, epoch, logs=None):
        self.start_time = time.time()
        self.batches_per_epoch = 0

    def on_train_batch_end(self, batch, logs=None):
        self.batches_per_epoch += 1

    def on_epoch_end(self, epoch, logs=None):
        epoch_time = time.time() - self.start_time
        # Calculate batches per second
        if epoch_time > 0:
            batches_per_sec = self.batches_per_epoch / epoch_time
        else:
            batches_per_sec = 0

        # Calculate progress percentage
        total_epochs = self.params.get('epochs', 1)
        progress = ((epoch + 1) / total_epochs) * 100

        # Prepare data to log with proper formatting
        epoch_num = epoch + 1
        progress_percent = f"{progress:.2f}"
        loss = f"{logs.get('loss', 0):.4f}"
        accuracy = f"{logs.get('accuracy', 0) * 100:.2f}"
        epoch_time_sec = f"{epoch_time:.2f}"
        batches_per_second = f"{batches_per_sec:.2f}"

        # Write to CSV
        with open(self.filename, 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([epoch_num, progress_percent, loss, accuracy, epoch_time_sec, batches_per_second])

# Train and evaluate the model
def train_and_evaluate_model(model, x_train, y_train, x_test, y_test, epochs=100, log_file=None, batch_size = 64):
    if log_file is None:
        # Generate a unique identifier for the log file
        unique_id = uuid.uuid4()
        log_file = f'../logs/training_log_tf_{unique_id}_bs_64_ep100.csv'

    # Initialize only ExtendedCSVLogger
    extended_csv_logger = ExtendedCSVLogger(log_file, append=False)  # Set append=False to start fresh

    start_time = time.time()
    model.fit(
        x_train,
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        verbose=2,  # You can set to 1 or 2 for different verbosity levels
        callbacks=[extended_csv_logger]
    )
    end_time = time.time()
    
    # Evaluate on test data
    loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"Test Loss: {loss:.4f}")
    print(f"Total Execution time: {end_time - start_time:.4f} seconds")
    return test_accuracy

# Main function to run the pipeline
def main(run_id):
    # Ensure the logs directory exists
    os.makedirs('../logs', exist_ok=True)
    batch_size = 128
    x_train, y_train, x_test, y_test = load_data()
    model = build_model()
    log_file = f'../logs_256_64/training_log_tf_{int(time.time())}_bs{str(batch_size)}_ep100.csv'
    train_and_evaluate_model(model, x_train, y_train, x_test, y_test, log_file=log_file, batch_size=batch_size)

if __name__ == "__main__":
    for i in range(100):
        main(i)


In [ ]:
import tensorflow as tf

# List physical devices available to TensorFlow
print("Available devices:")
for device in tf.config.list_physical_devices():
    print(device)

# Specifically check for GPUs
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("\nTensorFlow is using the GPU(s):")
    for gpu in gpus:
        print(gpu)
else:
    print("\nNo GPU detected, TensorFlow is using the CPU.")


In [ ]:
print("TensorFlow version:", tf.__version__)